In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns

from matplotlib.colors import LinearSegmentedColormap
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from matplotlib.colors import BoundaryNorm

#for server hpcr
import sys
sys.path.append("/home/jum002/store5/repo/smrt_fork/smrt")
sys.path.append("/home/jum002/code-workshop/radar_equivalent_snow/scripts")

#local import
import iso_function as iso


import warnings
warnings.filterwarnings('ignore')

In [2]:
xr_cdp = xr.open_dataset('~/store5/output/res_output/wfj_season_2013.nc')
df_cdp = xr_cdp.to_dataframe().dropna()

#get list of date
dates_cdp = df_cdp.groupby(level = 'time').mean().index.get_level_values(0)

In [3]:
MEPRA_dict = {'PP': 0,      # fr
              'PP+DF': 1,   # fr_lb
              'DF': 2,      # lb
              'DF+RG': 3,   # lb_fin
              'DF+FC': 4,   # lb_ang
              'PPgp': 5,    # roul
              'RG': 6,      # fin
              'MF+RG': 7,   # fin_ar
              'RG+FC': 8,   # fin_ang
              'FC': 9,      # pl
              'FC+DH': 10,  # pl_gob
              'DH': 11,     # gob
              'MF': 12,     # gel
              'MF+DH': 13,  # gob_fon
              'MF+FC': 14   # ron_ang
              }

MEPRA_labels = ['PP', 'PP+DF', 'DF', 'DF+RG', 'DF+FC', 'PPgp', 'RG', 'MF+RG', 'RG+FC', 'FC', 'FC+DH', 'DH', 'MF', 'MF+DH', 'MF+FC']

coloring = {'PP': np.array([0, 255, 0]) / 255.0,
            'MM': np.array([255, 215, 0]) / 255.0,
            'DF': np.array([34, 139, 34]) / 255.0,
            'RG': np.array([255, 182, 193]) / 255.0,
            'FC': np.array([173, 216, 230]) / 255.0,
            'DH': np.array([0, 0, 255]) / 255.0,
            'SH': np.array([250, 0, 255]) / 255.0,
            'MF': np.array([255, 0, 0]) / 255.0,
            'MFcr': np.array([255, 255, 255]) / 255.0,
            'IF': np.array([0, 255, 255]) / 255.0,
            'NO': np.array([200, 200, 200]) / 255.0}

color_grain = {MEPRA_dict['PP']: coloring['PP'],
               MEPRA_dict['PP+DF']: 0.5 * (coloring['PP'] + coloring['DF']),
               MEPRA_dict['DF']: coloring['DF'],
               MEPRA_dict['DF+RG']: 0.5 * (coloring['DF'] + coloring['RG']),
               MEPRA_dict['DF+FC']: 0.5 * (coloring['DF'] + coloring['FC']),
               MEPRA_dict['PPgp']: [0, 0, 0],
               MEPRA_dict['RG']: coloring['RG'],
               MEPRA_dict['MF+RG']: 0.5 * (coloring['MF'] + coloring['RG']),
               MEPRA_dict['RG+FC']: 0.5 * (coloring['RG'] + coloring['FC']),
               MEPRA_dict['FC']: coloring['FC'],
               MEPRA_dict['FC+DH']: 0.5 * (coloring['FC'] + coloring['DH']),
               MEPRA_dict['DH']: coloring['DH'],
               MEPRA_dict['MF']: coloring['MF'],
               MEPRA_dict['MF+DH']: 0.5 * (coloring['MF'] + coloring['DH']),
               MEPRA_dict['MF+FC']: 0.5 * (coloring['MF'] + coloring['FC'])}

grain_colormap = LinearSegmentedColormap.from_list("custom", [[i / 14., color_grain[i]] for i in range(15)])

In [ ]:
grain_colormap.N

In [5]:
# define the bins and normalize
bounds = np.linspace(0, len(MEPRA_dict), len(MEPRA_dict)+1)
norm = BoundaryNorm(bounds, grain_colormap.N)

In [6]:
def layer_k_noAverage(snow_df, n_clusters):
    
    X = pd.DataFrame({ 'ke' : iso.compute_ke(snow_df), 'height' : snow_df.height})
    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto").fit(X)
    snow_df['Group'] = kmeans.labels_
    snow_df['ke'] = iso.compute_ke(snow_df)

    return snow_df

In [ ]:
print(dates_cdp[150], dates_cdp[394])

In [8]:
#index choose a bit randomly to pick date early and mid season
snow_2_e = layer_k_noAverage(df_cdp.loc[dates_cdp[150],:], 2)
snow_2_m = layer_k_noAverage(df_cdp.loc[dates_cdp[394],:], 2)

snow_3_e = layer_k_noAverage(df_cdp.loc[dates_cdp[150],:], 3)
snow_3_m = layer_k_noAverage(df_cdp.loc[dates_cdp[394],:], 3)

In [ ]:
snow_3_e['h_btm'] = snow_3_e.height - snow_3_e.thickness
stack_h = np.dstack([snow_3_e.height, snow_3_e.h_btm]).flatten()
stack_d = np.dstack([snow_3_e.SNODEN_ML, snow_3_e.SNODEN_ML]).flatten()
stack_d

In [ ]:
snow_3_e

In [11]:
def change_group_e(row):
    if row['Group'] == 1:
        row['Group'] = 0
    elif row['Group'] == 0:
        row['Group'] = 1
    elif row['Group'] == 2:
        row['Group'] = 2
    return row

def change_group_m(row):
    if row['Group'] == 2:
        row['Group'] = int(0)
    elif row['Group'] == 0:
        row['Group'] = int(1)
    elif row['Group'] == 1:
        row['Group'] = int(2)
    return row

In [12]:
snow_3_e = snow_3_e.apply(lambda x: change_group_e(x), axis =1)
snow_3_m = snow_3_m.apply(lambda x: change_group_m(x), axis =1)

In [ ]:
snow_3_m.Group.astype(int)

In [ ]:
fig, ax = plt.subplots(2,5, figsize = (8,6), sharey = True, sharex = 'col', gridspec_kw={'width_ratios': [0.75, 0.75, 0.75, 0.75, 0.2]})

#snow1
#ax[0,2].set_title('WFJ : 2013-12-08')
sns.scatterplot(x = snow_2_e.ke, y= snow_2_e.height, hue = snow_2_e.Group, ax = ax[0,0], legend = True, palette= 'colorblind')
ax[0,0].set_ylabel('Height (m)')
ax[0,0].hlines(y = snow_2_e.SNODP[0]* 0.5, xmin=0, xmax=1, linestyles = ':', color = 'k', alpha = 0.5)
ax[0,0].set_xlabel('$\kappa_e$ ($m^{-1}$)')


sns.scatterplot(x = snow_3_e.ke, y= snow_3_e.height, hue = snow_3_e.Group.astype(int), ax = ax[0,1], legend = True, palette= 'colorblind')
ax[0,1].set_ylabel('Height (m)')
ax[0,1].hlines(y = snow_3_e.SNODP[0]* 0.33, xmin=0, xmax=1, linestyles = ':', color = 'k', alpha = 0.5)
ax[0,1].hlines(y = snow_3_e.SNODP[0]* 0.66, xmin=0, xmax=1, linestyles = ':', color = 'k', alpha = 0.5)

#ax[0,2].plot(snow_3_e.SNODEN_ML, snow_3_e.height, 'k', alpha = 0.5)
snow_3_e['h_btm'] = snow_3_e.height - snow_3_e.thickness
stack_h = np.dstack([snow_3_e.height, snow_3_e.h_btm]).flatten()
stack_d = np.dstack([snow_3_e.SNODEN_ML, snow_3_e.SNODEN_ML]).flatten()
stack_ssa = np.dstack([snow_3_e.ssa, snow_3_e.ssa]).flatten()

ax[0,2].plot(stack_d, stack_h, 'k', alpha = 0.5)
ax[0,3].plot(stack_ssa, stack_h, 'k', alpha = 0.5)
#ax[2].scatter(snow1.ssa, snow1.height, color = color_type)

#grain type
hgt_bot = snow_3_e.h_btm
hgt_top = snow_3_e.height
layers_obs = [Rectangle((0, h_bot), 1, h_top-h_bot,edgecolor=None) for h_bot,h_top in zip(list(hgt_bot),list(hgt_top))] 
# Create patch collection with specified colour/alpha
pc = PatchCollection(layers_obs, array = snow_3_e.SNOTYPE_ML, cmap=grain_colormap, norm = norm, 
                        edgecolor=None)
#ax[0,3].set_xticks([])
# Add collection to axes        
ax[0,4].add_collection(pc)  

ax[0,0].set_title('a)', size =10)
ax[0,1].set_title('b)', size =10)
ax[0,2].set_title('c)', size =10)
ax[0,3].set_title('d)', size =10)
#ax[0,4].set_title('e)', size =10)



#sno2
#ax[1,2].set_title('WFJ : 2014-02-09')
sns.scatterplot(x = snow_2_m.ke, y= snow_2_m.height, hue = snow_2_m.Group, ax = ax[1,0], palette= 'colorblind')
ax[1,0].set_ylabel('Height (m)')
ax[1,0].hlines(y = snow_2_m.SNODP[0]* 0.5, xmin=0, xmax=1, linestyles = ':', color = 'k', alpha = 0.5)
ax[1,0].set_xlabel('$\kappa_e$ ($m^{-1}$)')
ax[1,0].legend([]).remove()

sns.scatterplot(x = snow_3_m.ke, y= snow_3_m.height, hue = snow_3_m.Group.astype(int), ax = ax[1,1], palette= 'colorblind')
ax[1,1].set_ylabel('Height (m)')
ax[1,1].set_xlabel('$\kappa_e$ ($m^{-1}$)')
ax[1,1].hlines(y = snow_3_m.SNODP[0]* 0.33, xmin=0, xmax=1, linestyles = ':', color = 'k', alpha = 0.5)
ax[1,1].hlines(y = snow_3_m.SNODP[0]* 0.66, xmin=0, xmax=1, linestyles = ':', color = 'k', alpha = 0.5)
ax[1,1].legend([]).remove()



snow_3_m['h_btm'] = snow_3_m.height - snow_3_m.thickness
stack_h = np.dstack([snow_3_m.height, snow_3_m.h_btm]).flatten()
stack_d = np.dstack([snow_3_m.SNODEN_ML, snow_3_m.SNODEN_ML]).flatten()
stack_ssa = np.dstack([snow_3_m.ssa, snow_3_m.ssa]).flatten()

ax[1,2].plot(stack_d, stack_h, 'k', alpha = 0.5)
ax[1,2].set_xlabel('Density ($kg$ $m^{-3}$)')

ax[1,3].plot(stack_ssa, stack_h, 'k', alpha = 0.5)
ax[1,3].set_xlabel('SSA ($m^2$ $kg^{-1}$)')

ax[1,0].set_title('e)', size =10)
ax[1,1].set_title('f)', size =10)
ax[1,2].set_title('g)', size =10)
ax[1,3].set_title('h)', size =10)

#grain type
hgt_bot = snow_3_m.h_btm
hgt_top = snow_3_m.height
layers_obs = [Rectangle((0, h_bot), 1, h_top-h_bot,edgecolor=None) for h_bot,h_top in zip(list(hgt_bot),list(hgt_top))] 
# Create patch collection with specified colour/alpha
pc = PatchCollection(layers_obs, array = snow_3_m.SNOTYPE_ML, cmap=grain_colormap,  norm = norm,
                        edgecolor=None)
ax[1,4].set_xticks([])
# Add collection to axes        
ax[1,4].add_collection(pc)  

cax = ax[1,4].inset_axes([1.3,0,0.5,2.2])
cbar = plt.colorbar(pc, cax=cax)
labels = MEPRA_labels
cbar.set_ticks(np.arange(np.shape(labels)[0]) +0.5)
cbar.ax.set_yticklabels(labels)
ax[1,4].set_xlabel('\n        Grain type')
#cax.set_xlabel('\n Grain type')

plt.subplots_adjust(wspace = 0.05)
#plt.subplots_adjust(hspace = 0.1)


In [ ]:
150/7.5